In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np

In [3]:
features = pd.read_csv('CreacionDeFeatures/trainingSet.csv')

labels = pd.read_csv('CreacionDeFeatures/Data/labels_training_set.csv')

testing = pd.read_csv('CreacionDeFeatures/testingSet.csv')

In [4]:
features = features.set_index('person')

labels = labels.set_index('person')

testing = testing.set_index('person')

data = features.merge(labels, left_index=True, right_index=True)

data.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,VentasMes3,VentasMes4,PromedioVentasPorMes,label
person,,,,,,,,,,,,,,
0566e9c1,68,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.2,0
6ec7ee77,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
abe7a2fb,96,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
34728364,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
87ed62de,17,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
features, labels = data.iloc[:,:-1],data.iloc[:,-1]

In [6]:
features.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,VentasMes3,VentasMes4,PromedioVentasPorMes
person,,,,,,,,,,,,,
0566e9c1,68,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.2
6ec7ee77,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abe7a2fb,96,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34728364,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87ed62de,17,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [7]:
labels.head()

#Estan en el mismo orden

person
0566e9c1    0
6ec7ee77    0
abe7a2fb    0
34728364    0
87ed62de    0
Name: label, dtype: int64

In [8]:
data_dmatrix = xgb.DMatrix(data=features,label=labels)

In [9]:
params = {"objective":"binary:logistic",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 7, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="error", as_pandas=True, seed=123)

[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:05:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[

In [10]:
cv_results

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.050479,0.001933,0.050479,0.000966


In [68]:
xg_class = xgb.XGBClassifier(objective ='binary:hinge', min_child_weight = 100,
                colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 3, alpha = 10, n_estimators = 10)

In [69]:
xg_class.fit(features,labels)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=3, min_child_weight=100, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [70]:
preds = xg_class.predict(testing)

In [71]:
predicciones = pd.Series(preds)

predicciones

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
19385    0
19386    0
19387    0
19388    0
19389    0
19390    0
19391    0
19392    0
19393    0
19394    0
19395    0
19396    0
19397    0
19398    0
19399    0
19400    0
19401    0
19402    0
19403    0
19404    0
19405    0
19406    0
19407    0
19408    0
19409    0
19410    0
19411    0
19412    0
19413    0
19414    0
dtype: int64

In [72]:
predicciones.sum()

0